In [ ]:
    class OFDMSymbolType(enum.Enum):
        SYNC = 1
        ULP = 2
        ULD = 3
        GUARD = 4
        DLP = 5
        DLD = 6

In [1]:
def normlization(mat):
    
    mat = np.array(mat['channels_write'])
    #normalization
    factor_raw = 0
    for k in range(0,Nsub):
        factor_raw = factor_raw + np.square(np.linalg.norm(mat[k,:,:]))
    factor = 1/np.sqrt(factor_raw/(M*K*Nsub))
    H_norm = np.full_like(mat,0)
    for k in range(0,Nsub):
        for i in range(0,M):
            for j in range(0,K):
                H_norm[k,i,j] = mat[k,i,j]*factor
    return H_norm

In [ ]:
def addnoise(inputVector1,currentSymbolType,target_snr_db):
    inputVector_n = inputVector1
    if currentSymbolType == OFDMSymbolType.ULD:
        signal_power = 10*np.log10(np.square(np.linalg.norm(inputVector1)))
        noise_power = 10**((signal_power - target_snr_db)/10)
        noise = np.random.normal(0, 1, size=(M,2)).view(complex)
        noise_real_power = np.square(np.linalg.norm(noise))
        noise1 = noise*np.sqrt(noise_power/noise_real_power)
        inputVector_n = inputVector1 + noise1
    return inputVector_n

In [ ]:
def findNearestVector(inputVector,possibleSymbol,H):
    global radius   
    d = [] 

    for i in range(0,K):
        d.append(possibleSymbol[i,0:radius[i]])
    possibleVector = np.array(list(itertools.product(*d)))/math.sqrt(10)
    distance = 10000000
    for i in range(0,possibleVector.size//K):
        tmp_dis = np.sum(abs(np.dot(H,possibleVector[i])-inputVector.reshape(M))**2)
        
        if tmp_dis<distance:
            distance = tmp_dis
            smallest = possibleVector[i]
    return smallest

In [ ]:
def findNearest(symbolInput,Nbits,Npower):
    global Symbol4Function
    Symbol4Function = symbolInput*math.sqrt(Npower)
    symbolsOutput = np.array([0+0j for _ in range(2**Nbits)])
    if Nbits ==4 :
        symbolsOutput = sorted(np.array(QAM16Symbols), key=findDistance_QAM16)

    return symbolsOutput

In [ ]:
def findDistance_QAM16(symbolIn):
    global Symbol4Function
    return abs(symbolIn-Symbol4Function)